In [2]:
import praw
import pandas as pd

In [61]:
# get reddit secret & script i've saved as .txt in the working directory

with open('reddit_oa_script.txt') as f:
    
    reddit_script = f.read().strip()
    
with open('reddit_oa_secret.txt') as f:
    
    reddit_secret = f.read().strip()
    
with open('reddit_id.txt') as f:
    
    pw = f.read().strip()

In [62]:
# user_agent is the name of the app



reddit  = praw.Reddit(client_id=reddit_script,
                          client_secret=reddit_secret,
                          user_agent='get_corpora',
                          username='neurodivergent_ai',
                          password=pw)


In [63]:
# get the subreddit

subreddit_1 = reddit.subreddit('The_Donald')

# get the top threads

subreddit_1_top = subreddit_1.top(limit=1000)

In [64]:
for submission in subreddit_1.top(limit=2):
    print(submission.title, submission.id)

DONALD J. TRUMP DECLARED THE WINNER! 5bzjv5
I’m Donald J. Trump and I'm Your Next President of the United States. 4uxdbn
